# After training for the first run, we load model

In [2]:
from  hubconf import custom
import numpy as np
model = custom(path_or_model='runs/train/yolo_ball_det/weights/best.pt')  # custom example



                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


# check performance on test&val set

In [14]:
import os
import  glob
def check_performance(root, thres=None, return_high_conf_samples=True):
    conf_li =[]
    collect_sample_li = []#storing high conf samples
    for img_pth in glob.glob(root+'*.jpg'):
        results = model(img_pth)  # batched inference
        results_df = results.pandas().xyxy[0]
        conf = results_df['confidence']
        try:
            conf = float(conf[0])
            ID = img_pth.split('/')[-1].split('.')[0]
            if thres:
                if conf>=thres:
                    conf_li.append(conf)
                    collect_sample_li.append(ID)
            else:
                conf_li.append(conf)
                collect_sample_li.append(ID)
        except:
            #cannot detect anything, skip to the next one!
            continue
            
    print('='*40)
    s = pd.Series(conf_li)
    print('confidence distribution: ')
    print(s.describe())
    
    if return_high_conf_samples:
        return collect_sample_li

In [41]:
# we can see our model can do better on both test and val
import warnings
root = 'custom_data/train_val_test/images/test/'
warnings.filterwarnings('ignore') 
check_performance(root, thres=None, return_high_conf_samples=False)

confidence distribution: 
count    28.000000
mean      0.311248
std       0.050038
min       0.253004
25%       0.266991
50%       0.299716
75%       0.340887
max       0.405409
dtype: float64


In [46]:
root = 'custom_data/train_val_test/images/val/'
warnings.filterwarnings('ignore') 
check_performance(root, thres=None, return_high_conf_samples=False)

confidence distribution: 
count    32.000000
mean      0.308756
std       0.045300
min       0.252418
25%       0.273310
50%       0.295162
75%       0.339335
max       0.420723
dtype: float64


# check performance on pseudo_annotated set

In [58]:
root = 'custom_data/pseudo_annotated/images/'
warnings.filterwarnings('ignore') 
collect_sample_li = check_performance(root, thres=None, return_high_conf_samples=True)

confidence distribution: 
count    274.000000
mean       0.287035
std        0.029999
min        0.250189
25%        0.263778
50%        0.279195
75%        0.300628
max        0.392158
dtype: float64


In [65]:
# because the other samples come from different video shots, their confidence can be low.
# we can see that there only some samples were detected by the model
# applying a confidence threshold might not serve well here
print('number of pseudo_annotated samples: {0}'.format(len(glob.glob(root+'*.jpg'))))
print('number of samples (with/without) target object detected by the model: {0}'.format(len(conf_li)))

number of pseudo_annotated samples: 1584
number of samples (with/without) target object detected by the model: 277


In [ ]:
# since the minimum&maximum confidence are not too different, 
# we'll move all image samples shown in conf_li to the train set

# move detected sample from pseudo_annotated set to train set

In [66]:
pseudo_root = 'custom_data/pseudo_annotated/'
train_val_test_root = 'custom_data/train_val_test/'
for ID in collect_sample_li:
    img_src = pseudo_root+'images/'+ID+'.jpg'
    img_dst = train_val_test_root+'images/train/'+ID+'.jpg'
    os.rename(img_src, img_dst)
    
    txt_src = pseudo_root+'labels/'+ID+'.txt'
    txt_dst = train_val_test_root+'labels/train/'+ID+'.txt'
    os.rename(txt_src, txt_dst)

# open the terminal and start another run of training

using the same command but modify starting weights to ```runs/train/yolo_ball_det/weights/best.pt``` and weight folder name to ```yolo_ball_det_2```:
    
```python train.py --img-size 640 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 16 --epochs 20 --data custom_data/dataset.yaml --weights runs/train/yolo_ball_det/weights/best.pt --workers 24 --name yolo_ball_det_2``` 

# check performance on test&val set

In [3]:
from  hubconf import custom
import numpy as np
new_round = 'yolo_ball_det_2'
model = custom(path_or_model='runs/train/'+new_round+'/weights/best.pt')


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


In [68]:
root = 'custom_data/train_val_test/images/test/'
warnings.filterwarnings('ignore') 
check_performance(root, thres=None, return_high_conf_samples=False)

confidence distribution: 
count    84.000000
mean      0.629820
std       0.112759
min       0.262919
25%       0.558461
50%       0.617230
75%       0.699239
max       0.845739
dtype: float64


In [70]:
root = 'custom_data/train_val_test/images/val/'
warnings.filterwarnings('ignore') 
check_performance(root, thres=None, return_high_conf_samples=False)

confidence distribution: 
count    84.000000
mean      0.667627
std       0.100580
min       0.471805
25%       0.590472
50%       0.652266
75%       0.758371
max       0.874292
dtype: float64


# The mean confidence shifted from 30%+- to 66%+-, a big jump! 

Note that the count is changed here, it is because undetected samples were not counted. In the first run, counted samples was a small number. As the model improves, it detects target objects in more samples. Thus, it increases the count.


# check performance on pseudo_annotated set

In [5]:
import warnings
import pandas as pd
root = 'custom_data/pseudo_annotated/images/'
warnings.filterwarnings('ignore') 
collect_sample_li = check_performance(root, thres=None, return_high_conf_samples=True)

confidence distribution: 
count    1046.000000
mean        0.546904
std         0.112113
min         0.250351
25%         0.482870
50%         0.558698
75%         0.624349
max         0.792934
dtype: float64


# Detected count in pseudo_annotated set improves from 274 to 1046!

### Let's add more data to the train set

In [6]:
# the same piece of code
def move_data(collect_sample_li, pseudo_root='custom_data/pseudo_annotated/', train_val_test_root='custom_data/train_val_test/'):
    for ID in collect_sample_li:
        img_src = pseudo_root+'images/'+ID+'.jpg'
        img_dst = train_val_test_root+'images/train/'+ID+'.jpg'
        os.rename(img_src, img_dst)

        txt_src = pseudo_root+'labels/'+ID+'.txt'
        txt_dst = train_val_test_root+'labels/train/'+ID+'.txt'
        os.rename(txt_src, txt_dst)

In [7]:
move_data(collect_sample_li)

In [10]:
len(collect_sample_li)

1046

### Let's train the model again

using the same command but modify starting weights to ```runs/train/yolo_ball_det_2/weights/best.pt``` and weight folder name to ```yolo_ball_det_3```:
    
```python train.py --img-size 640 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 16 --epochs 20 --data custom_data/dataset.yaml --weights runs/train/yolo_ball_det_2/weights/best.pt --workers 24 --name yolo_ball_det_3``` 

### Let's inferencing the model with test set

In [8]:
new_round = 'yolo_ball_det_3'
model = custom(path_or_model='runs/train/'+new_round+'/weights/best.pt')


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


In [9]:
root = 'custom_data/train_val_test/images/test/'
warnings.filterwarnings('ignore') 
check_performance(root, thres=None, return_high_conf_samples=False)

confidence distribution: 
count    84.000000
mean      0.670466
std       0.107864
min       0.306934
25%       0.606582
50%       0.663608
75%       0.739681
max       0.870704
dtype: float64


# Oh no! After we added 1046 images to train set, we only get 4% improvement on mean confidence in the test set!

### Reverse moved data: move back data defined in collect_sample_li from train set to pseudo_annotated set

In [11]:
def reverse_data(collect_sample_li, pseudo_root='custom_data/pseudo_annotated/', train_val_test_root='custom_data/train_val_test/'):
    for ID in collect_sample_li:
        img_dst = pseudo_root+'images/'+ID+'.jpg'
        img_src = train_val_test_root+'images/train/'+ID+'.jpg'
        os.rename(img_src, img_dst)

        txt_dst = pseudo_root+'labels/'+ID+'.txt'
        txt_src = train_val_test_root+'labels/train/'+ID+'.txt'
        os.rename(txt_src, txt_dst)

In [12]:
reverse_data(collect_sample_li)

### This time, we will set a confidence THRESHOLD and filter out pseudo_annotated data that has low confidence! The threshold = top 75% confidence == 0.7 of the second run

In [16]:
root = 'custom_data/pseudo_annotated/images/'
warnings.filterwarnings('ignore') 
collect_sample_li = check_performance(root, thres=0.7, return_high_conf_samples=True)

confidence distribution: 
count    141.000000
mean       0.733813
std        0.026603
min        0.700623
25%        0.710778
50%        0.732635
75%        0.745979
max        0.824090
dtype: float64


# Train the model again, and hope adding small data will still help! (fingers crossed!)

using the same command as the third run but change --name to ```yolo_ball_det_3_thres```:
    
```python train.py --img-size 640 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 16 --epochs 20 --data custom_data/dataset.yaml --weights runs/train/yolo_ball_det_2/weights/best.pt --workers 24 --name yolo_ball_det_3_thres``` 

### Let's inferencing the model with test set

In [17]:
new_round = 'yolo_ball_det_3_thres'
model = custom(path_or_model='runs/train/'+new_round+'/weights/best.pt')



                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


In [18]:
root = 'custom_data/train_val_test/images/test/'
warnings.filterwarnings('ignore') 
check_performance(root, thres=None, return_high_conf_samples=False)

confidence distribution: 
count    84.000000
mean      0.740976
std       0.094342
min       0.390799
25%       0.680425
50%       0.738316
75%       0.801788
max       0.902722
dtype: float64


# After we added 141 images to train set, the model got 11% (0.74-0.63) improvement on mean confidence in the test set!

# we can run test results in the terminal
```python test.py --weights runs/train/yolo_ball_det_3_thres/weights/best.pt --data custom_data/dataset.yaml --task test --name yolo_det_3_thres```


| run     | P     | R     | mAP@.5 | mAP@.5:.95 | added data from the 2nd run |
|---------|-------|-------|--------|------------|-----------------------------|
| 3       | 0.974 | 0.894 | 0.945  | 0.825      | 1046                        |
| 3+thres | 0.974 | 0.906 | 0.951  | 0.82       | 141                         |

### we get similar performance on mAP comparing to the third run without threshold, despite we added MUCH SMALLER dataset to the trainset

# This has told us that more data does not promise a peformance boost, especially when data labels isn't verified by human

### What's Next?
1. We can see improvement is decreasing, thus, adding samples from pseudo_annotated set might be needed to increase model generalization if we can verify label quality
2. We need to be aware that adding small datasets for retraining a custom model for many times might lose generalization ability of the model, we can consider only retrain model head (classification part) or a small portion of the model parameters.